In [44]:
# read data
import pandas as pd
import data_utils as du

path_to_media = './data/extra_infos.json'
path_to_train = './data/train.csv'

media_info = du.json_to_pandas(path_to_media)
train = pd.read_csv(path_to_train, encoding='utf-8', usecols=['genre_id', 'media_id', 'album_id', 'release_date', 'media_duration', 'artist_id'])

In [45]:
# merge data
media = pd.merge(train, media_info, on='media_id')
media.head()

,genre_id,media_id,album_id,release_date,media_duration,artist_id,alb_title,art_name,sng_title
0,25471,222606,41774,20040704,223,55164,Malin plaisir,Malin Plaisir,Seul au monde
1,25571,250467,43941,20060301,171,55830,Stop talking / dead tapes,Soldout,You're not ok
2,16,305197,48078,20140714,149,2704,Dérives,René Aubry,Samuel
3,7,900502,71521,20001030,240,938,Estilolibre,Eros Ramazzotti,Mujer Amiga Mia
4,7,542335,71718,20080215,150,2939,Collections - The Byrds Play Dylan,The Byrds,Mr. Tambourine Man


In [46]:
# select all artists
by_artist = media.groupby(['artist_id'], as_index=False).agg({'art_name' : 'first'})
by_artist["name"] = by_artist["art_name"]
del by_artist["art_name"]
by_artist.head()

,artist_id,name
0,1,The Beatles
1,2,The Clash
2,3,Snoop Dogg
3,4,America
4,5,Amadou & Mariam


Python driver для Neo4j - гівно, працює дуже повільно. Буду імпортувати csv. Але перед цим мені потрібно зберегти ці значення.

In [38]:
by_artist.to_csv('./data/graph/import/artists.csv', index=False)

USING PERIODIC COMMIT 5000

LOAD CSV WITH HEADERS FROM "file:///artists.csv" AS row 

MERGE (artist:Artist {art_id: toInt(row.artist_id)})

ON CREATE SET artist.art_id = toInt(row.artist_id), artist.name = row.name

CREATE CONSTRAINT ON (a:Artist) ASSERT a.art_id IS UNIQUE

Артисти є. Що ще треба завантажити:

1. Альбоми. Зв'язки між виконавцями та альбомами. (done)

2. Пісні. Зв'язки між альбомами та піснями (done)

3. Користувачів. (done)

4. Прослуховування - зв'язок між користувачем і піснею. (done)

5. Жанри. Зв'язок між жанром і піснею. (done)

In [39]:
by_album = media.groupby(['album_id'], as_index=False).agg({'alb_title' : 'first', 'artist_id':'first'})
by_album.head()

,album_id,artist_id,alb_title
0,1976,443,Terrain vague
1,4096,443,Croc' noces
2,6108,443,Fausses notes et repris de justesse
3,6558,443,"Irfan, le héros"
4,7782,443,Rue du temps


In [40]:
by_album.to_csv('./data/graph/import/albums.csv', index=False)

Думаю, який обрати relation_type. Тобто це ж має бути не з реального світу, а те що мені підходить. Типу ART_RELATED

https://neo4j.com/docs/developer-manual/current/get-started/cypher/importing-csv-files-with-cypher/
    
http://neo4j.com/docs/developer-manual/current/cypher/clauses/load-csv/

https://neo4j.com/blog/bulk-data-import-neo4j-3-0/

USING PERIODIC COMMIT 10000

LOAD CSV WITH HEADERS FROM "file:///albums.csv" AS row 

MERGE (album:Album {alb_id: toInt(row.album_id)}) 

ON CREATE SET album.alb_id = toInt(row.album_id), album.name = row.alb_title

WITH row, album

MATCH (art:Artist)

WHERE art.art_id = toInt(row.artist_id)

MERGE (album)-[:ART_RELATED]-(art)

CREATE CONSTRAINT ON (a:Album) ASSERT a.alb_id IS UNIQUE

In [49]:
by_media = media.groupby(['media_id'], as_index=False).agg({'sng_title' : 'first', 'album_id':'first'})
by_media.head()

,media_id,sng_title,album_id
0,200058,À saint nazaire y'a t'une brune,40004
1,200091,Robert et josiane,40006
2,200098,2 bi 3,40006
3,200099,Blanche neige et les 7 scouts,40006
4,200102,Mais qu'est-ce qui se passe ?,40006


In [43]:
by_media.to_csv('./data/graph/import/media.csv', index=False)

USING PERIODIC COMMIT 30000

LOAD CSV WITH HEADERS FROM "file:///media.csv" AS row

MERGE (media:Media {media_id: row.media_id})

ON CREATE SET media.media_id = row.media_id, media.name = row.sng_title

WITH row, media

MATCH (alb:Album)

WHERE alb.alb_id = toInt(row.album_id)

MERGE (media)-[:ART_RELATED]-(album)

Чогось зависає останній запис. Спробую запустити його по частинах, спочатку створення вершини, а потім створення зв'язку.

USING PERIODIC COMMIT 30000

LOAD CSV WITH HEADERS FROM "file:///media.csv" AS row

MERGE (media:Media {media_id: row.media_id})

ON CREATE SET media.media_id = row.media_id, media.name = row.sng_title

USING PERIODIC COMMIT 10000

LOAD CSV WITH HEADERS FROM "file:///media.csv" AS row

MATCH (album:Album),(media:Media)

WHERE album.alb_id = toInt(row.album_id) AND media.media_id = row.media_id

MERGE (media)-[:ART_RELATED]-(album)

CREATE CONSTRAINT ON (a:Media) ASSERT a.media_id IS UNIQUE

Зроблю ще жанри, щоб не перекачувати датасет і перейду до користувачів

In [54]:
by_genre = media.groupby(['media_id'], as_index=False).agg({'genre_id' : 'first'})
by_genre.head()

,media_id,genre_id
0,200058,25471
1,200091,25471
2,200098,25471
3,200099,25471
4,200102,25471


In [52]:
by_genre.to_csv('./data/graph/import/genres.csv', index=False)

Тепер буду завжди розділяти створення вершин і ребер при імпорті щоб простіше було дебажити.

USING PERIODIC COMMIT 30000

LOAD CSV WITH HEADERS FROM "file:///genres.csv" AS row

MERGE (genre:Genre {genre_id: row.genre_id})

ON CREATE SET genre.genre_id = row.genre_id

Чого я так тупанув і включив колонку `media_id` взагалі? Треба було спочатку створити жанри, а потім вже додати до датасету медіа жанр і створювати рілейшени.

USING PERIODIC COMMIT 30000

LOAD CSV WITH HEADERS FROM "file:///genres.csv" AS row

MATCH (genre:Genre),(media:Media)

WHERE genre.genre_id = row.genre_id AND media.media_id = row.media_id

MERGE (media)-[:GENRE_IS]->(genre)

CREATE CONSTRAINT ON (a:Genre) ASSERT a.genre_id IS UNIQUE

In [3]:
import pandas as pd

path_to_train = './data/train.csv'

train = pd.read_csv(path_to_train, encoding='utf-8', usecols=['user_id', 'media_id', 'is_listened', 'ts_listen' ])

Я думаю, потім можна додати зв'язок `is_not_listened`. Але поки що буде досить взяти всі `IS_LISTENED`. Це і так дофіга данних.

In [59]:
by_user = train.groupby('user_id', as_index=False).first()
by_user.head()

,user_id,media_id,is_listened
0,0,17390736,1
1,1,1109488,1
2,2,2438245,1
3,3,983205,1
4,4,876497,1


In [60]:
by_user.to_csv('./data/graph/import/users.csv', index=False)

USING PERIODIC COMMIT 10000

LOAD CSV WITH HEADERS FROM "file:///users.csv" AS row

MERGE (user:User {user_id: row.user_id})

ON CREATE SET user.user_id = row.user_id

CREATE CONSTRAINT ON (a:User) ASSERT a.user_id IS UNIQUE

In [4]:
listened = train[train['is_listened'] == 1].sort_values(by='ts_listen').groupby(['user_id', 'media_id'],as_index=False).agg({'ts_listen':'first'})
listened.head()

,user_id,media_id,ts_listen
0,0,211678,1479414226
1,0,211796,1479971016
2,0,238286,1480576692
3,0,286191,1480153220
4,0,304849,1478078842


In [5]:
listened.to_csv('./data/graph/import/listened.csv', index=False)

USING PERIODIC COMMIT 50000

LOAD CSV WITH HEADERS FROM "file:///listened.csv" AS row

MATCH (user:User), (media:Media)

WHERE user.user_id = row.user_id AND media.media_id = row.media_id

MERGE (user)-[:LISTENED {ts_listened: row.ts_listen}]->(media)